
# Italian Covid-19 Analysis

This notebook analyses data about covid-19 released by the Italian Protezione Civile and builds a predictor for the end of the epidemics. The general concepts behind this predictor are described in the following article: https://medium.com/@angelica.loduca/predicting-the-end-of-the-coronavirus-epidemics-in-italy-8da9811f7740.

The notebook exploits the `pandas` and `scikit-learn` libraries.


## Import data
Firstly, we import data from the Github repository of the Italian Protezione Civile and then we calculate the Epidemics Progression Index (EPI). We extract columns `totale_casi`, which contains the total number of covid-19 infections since the epidemics began, and `tamponi`, which contains the total number of covid-19 swabs since the epidemics began. We store EPI in the `y` variable. Finally, we print all the list of dates for which we have data (`data['data']`).

In [ ]:
import pandas as pd 

data = pd.read_csv("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv") 
tc = data['totale_casi']
tt = data['tamponi']
y = []
tt_increase = []
for i in range(1, len(tt)):
    current_epi = (tc[i] - tc[i-1])/(tt[i]-tt[i-1])*100
    tt_increase.append(tt[i]-tt[i-1])
    y.append(current_epi)
data['data']

Now we calculate the days for which we have data. We could use the variable `data['data']`. However, this variable is a string, so it is difficult to use it to fit a predictor. Thus we use a generic variable `X`, which contains a sequence of numbers, from 1 to the total number of samples.

In [ ]:
X = []
for i in range(1, len(y)+1):
    X.append([i])

X

## Preliminary plot
Now we can plot y versus X, which means plotting EPI versus days. We plot also 2 additional lines: 1) the date corresponding to the beginning of restriction laws, 2) the date when the curve begins to decrease (a week later). These two lines can be understood when the plot is showed.

In Italy restrictions laws began March 9th, which corresponds to the 14th day. In other words it corresponds to the 14 row in the `y` and `X` variables. Effects of the restriction laws can be seen after a week from March 9th. Thus they began on the 21st day.

In [ ]:
# vertical line corresponding to the beginning of restriction laws. 
di = 14
restrictions_x = [di,di,di,di,di,di]
restrictions_y = [0,10,20,30,40,50]

In [ ]:
# vertical line corresponding to the beginning of effects of restriction laws (after a week)
de = di + 7
effects_x = [de,de,de,de,de,de]
effects_y = [0,10,20,30,40,50]
de

Now we can plot the graph. We can convert `X` values to dates through the `xticks()` function.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(X, y,  color='black')
plt.plot(restrictions_x,restrictions_y, color='red', linewidth=2)
plt.plot(effects_x,effects_y, color='green', linewidth=2)
plt.grid()
plt.xlabel('Days')
plt.xlim(0,40)
plt.ylim(0,50)
plt.xticks([0,5,10,15,20,25,30,35,40],
           ["24 Febr", "29 Febr", "5 Mar", "10 Mar", "15 Mar", "20 Mar", "25 Mar", "30 Mar", "4 Apr"])

plt.ylabel('Epidemics Progression Index (EPI)')
plt.savefig("EPI-all.png")
plt.show()

We note that EPI begins to decrease from March 16th, i.e. exactly a week after restriction laws. Thus, we can approximate the curve from March 16th with a linear regression, which decreases progressively. 

## Build the model

From the `sklearn` library we import the `linear_model`. Then we drop from `X` and `y` data before March 16th and we fit the `LinearRegression` model with `X` and `y`. Finally, we calculate the score of the model through the `score()` function.

In [ ]:
import numpy as np
from sklearn import linear_model

# alleno il modello solo a partire dagli effetti del cambiamento
X = X[de:]
y = y[de:]

print(X)
# Linear Regression
linear_regr = linear_model.LinearRegression()

# Train the model using the training sets
linear_regr.fit(X, y)


linear_regr.score(X,y)

## Predict future trend
Once fitted, we build the linear curve representing data, through function `predict()` and we calculate the maximum error done by the model through the function `max_error()` applied to the real values of `y` and the predicted values `y_pred`. This error will be used to build two new lines, the max line and minimum line, which will define the ranges between which new samples will move.

In [ ]:
from sklearn.metrics import max_error
import math

y_pred = linear_regr.predict(X)
error = max_error(y, y_pred)
error

Now we can apply the model to predict EPI for next days. We build a variable `X_test`, which contains both old days (i.e. days for which data already are available) and future days. We define the variable `gp` containing the number of previsions days. 

Then we apply our model to `X_test`.

In [ ]:
X_test = []

gp = 40

for i in range(de, de + gp):
    X_test.append([i])

y_pred_linear = linear_regr.predict(X_test)

The variable `y_pred_linear` contains the predicted IPE for next days. However, we should consider also the error made by the model. Thus we build two new variables `y_pred_max` and `y_pred_min` containing the `y_pred + error` and `y_pred - error`, respectively.

In [ ]:
y_pred_max = []
y_pred_min = []
for i in range(0, len(y_pred_linear)):
    y_pred_max.append(y_pred_linear[i] + error)
    y_pred_min.append(y_pred_linear[i] - error)

## Plot results
Now we are ready to plot data (`y_pred`, `y_pred_max` and `y_pred_min`). In order to make the plot more appealing, we should convert numbers (represented by the `X_test` variable) to dates. Thus we maintain two variables `x_ticks` and `date_prev`, which contain a subset of `X_test` and corresponding labels, respectively.. 

To build the mapping between `x_ticks` and `date_prev`, we extract the date of restrictions from the variable `data['data']` and we convert it to a date through the function `datetime.strptime()`. We store results in the variable `data_eff`. 

We build `x_ticks`, by sampling `X_test` every `step` items. We define `date_prev[0] = data_eff`. For each item, we can build `date_prev` by adding the `step` to the previous item.

In [ ]:
# calcolo la data iniziale degli effetti del cambiamento
from datetime import datetime
from datetime import timedelta  

data_eff = datetime.strptime(data['data'][de], '%Y-%m-%dT%H:%M:%S')
# date previsione
date_prev = []
x_ticks = []
step = 5
data_curr = data_eff
x_current = de
n = int(gp/step)
for i in range(0, n):
    date_prev.append(str(data_curr.day) + "/" + str(data_curr.month))
    x_ticks.append(x_current)
    data_curr = data_curr + timedelta(days=step)
    x_current = x_current + step


Now we can plot all the lines.

In [ ]:
plt.grid()
plt.scatter(X, y,  color='black')

plt.plot(X_test, y_pred_linear, color='green', linewidth=2)
plt.plot(X_test, y_pred_max, color='red', linewidth=1, linestyle='dashed')
plt.plot(X_test, y_pred_min, color='red', linewidth=1, linestyle='dashed')

plt.xlabel('Days')
plt.xlim(de,de+gp)

plt.xticks(x_ticks, date_prev)
plt.ylabel('Epidemics Progression Index (EPI)')
plt.yscale("log")

plt.savefig("EPI-prediction.png")
plt.show()

## Prediction of the 0 value of EPI

Firstly we calculate an auxiliary function, which converts a number to the date. This function considers the as starting date `data_eff`.

In [ ]:
def n_to_date(n):
    return data_eff + timedelta(days=n-de)

Now we can calculate when the line crosses the x axis. Remind that the equation of a line is y = ax + b and the equation of the x axis is y = 0. Thus we have to solve the system containing both equations. As a result x = -b/a.

In [ ]:
data_zero = round(- linear_regr.intercept_ / linear_regr.coef_[0])
n_to_date(data_zero)

The previous calculus is done only for the line defined by the regression. Now we should calculate also values for `y_pred_max` and `y_pred_min`. We define an auxiliary function, called `build_line()`, which builds a line from two points and returns the coefficients of the line.

In [ ]:
def build_line(x1,y1,x2,y2):
    m = float(y2 - y1)/(x2-x1)
    q = y1 - (m*x1)
    return [m,q]

Now we can calculate the ending date for `y_pred_min` and `y_pred_max`. We approximate the zero date for max to the `ceil()` of obtained value, and the zero date for min to the `floor()` of the obtained value.

In [ ]:
import math
line_max = build_line(X_test[0][0], y_pred_max[0], X_test[1][0], y_pred_max[1])
data_zero_max = math.ceil(- line_max[1] / line_max[0])
n_to_date(data_zero_max)

In [ ]:
line_min = build_line(X_test[0][0], y_pred_min[0], X_test[1][0], y_pred_min[1])
data_zero_min = math.floor(- line_min[1] / line_min[0])
n_to_date(data_zero_min)

## Predict the value of EPI for a generic date

We define an auxiliary function, called `date_to_n()`, which converts a generic date to the number of days since starting date. Then, we apply the model to the obtained value.

In [ ]:
def date_to_n(my_date):
    initial_date = datetime.strptime(data['data'][0], '%Y-%m-%dT%H:%M:%S')
    return (my_date - initial_date).days + 1

my_date = datetime.strptime("2020-04-05", '%Y-%m-%d')
n = date_to_n(my_date)
predict = linear_regr.predict([[n]])
predict[0]

## Predicting the maximum number of infections
Now we can build the maximum values of the total number of infections. We can exploit the formula of EPI: `y_pred[i] = (tc[i] - tc[i-1])/(tt[i]-tt[i-1])*100` to calculate the value of `tt[i] = y_pred[i]*/(tt[i]-tt[i-1])*100) + tc[i]`. The difference `tt[i]-tt[i-1]` represents the number of swabs at instant `i`, thus we can approximate it as an average value of `data['tamponi']` (stored in the variable `tt_increase`). Alternatively, we could calculate also the difference as the minimum or maximum value. Thus, we calculate the average value of `tt` (called `avg_tt`) as a metrics of `tt_increase[de:]`. The metrics is passed as parameter of the function.

In [ ]:
def average(mylist):
    return sum(mylist)/len(mylist)

# calculate the plateau considering the average increase of swabs
def plateau(y_pred,data_end,metrics):
    avg_tt = metrics(tt_increase[de:])

    np_avg = []
    #np_start = data['totale_casi'][len(data['totale_casi'])-1]
    np_start = data['totale_casi'][de]
    np_avg.append(np_start)

    for i in range(0, data_end-de):
        np = y_pred[i]*avg_tt/100 + np_avg[i-1]
        np_avg.append(np)
        
    last_value = max(np_avg)
    for i in range(0, gp-len(np_avg)):
        np_avg.append(last_value)
    return np_avg

Now, we can calculate the plateau for `y_pred_min`, `y_pred_max` and `y_pred_linear` by considering the maximum as metrics.

In [ ]:
plateau_min = plateau(y_pred_min,data_zero_min, max)
plateau_max = plateau(y_pred_max,data_zero_max, max)
plateau_avg = plateau(y_pred_linear,int(data_zero), max)

Finally, we plot results and we print the maximum values.

In [ ]:
plt.plot(X_test,plateau_min, color='red', linewidth=1, linestyle='dashed')
plt.plot(X_test,plateau_max, color='red', linewidth=1, linestyle='dashed')
plt.plot(X_test,plateau_avg, color='green', linewidth=2)
plt.scatter(X,tc[de+1:], color='black', linewidth=2)
plt.xlabel('Days')
plt.xlim(de,de+gp)
#plt.ylim(0,50)
plt.xticks(x_ticks, date_prev)
#plt.yticks([0,20,30,40,50,60])

plt.ylabel('Total number of positives')
plt.grid()
plt.savefig("TNP.png")
plt.show()


In [ ]:
max(plateau_min)

In [ ]:
max(plateau_max)

In [ ]:
max(plateau_avg)